<a href="https://colab.research.google.com/github/anuragiiser/Contract_review/blob/main/Clause_classification_Bioler_plate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Doc2Vec added, some more changes needed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install import-ipynb
!pip install python-docx
!pip install python-terrier
!pip install XlsxWriter
!pip install git+https://github.com/LexPredict/lexpredict-lexnlp.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=4e2a1170fa8d7d483ed12dc306e3504e07cdd4e528366d0ecc20b1fcd021bd29
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/GarimaDevnani/AI-Contract-Review

Cloning into 'AI-Contract-Review'...
remote: Enumerating objects: 288, done.
remote: Counting objects: 100% (288/288), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 288 (delta 59), reused 288 (delta 59), pack-reused 0
Receiving objects: 100% (288/288), 8.07 MiB | 4.42 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [ ]:
import import_ipynb
import docx
import pandas as pd

import docx
from docx import Document

import re
import spacy
nlp = spacy.load("en_core_web_sm")

import os

import pyterrier as pt

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import lexnlp.extract.ml.en.definitions as definitions
#!0pip install Jinja2
#help(lexnlp.extract.ml.en.definitions.tests)

#import pandas.io.formats.style
from pandas import ExcelWriter
import numpy as np

from xlsxwriter.utility import xl_rowcol_to_cell

from datetime import datetime

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
def preprocessing_master(master_df):
    df = master_df.copy()
    
    df["sub_category"] = df["sub_category"].str.replace('\n', '')
    df["clause"] = df["clause"].str.replace('\n', ' ')
    df = df.replace(r'[^0-9a-zA-Z ]', ' ', regex=True)
    df = df[df['clause'] != '']
    df.reset_index(drop=True, inplace=True)

    df['tokens'] = df['clause'].apply(tokenize)

    return df 


def tokenize(text):
    token_texts = []
    tokens = nlp(text)
    for token in tokens:
        if not token.is_stop and not token.is_punct and not token.is_space:
            token_texts.append(token.lemma_.lower())
    return " ".join(token_texts)


def preprocessing_contract(df):
    contract_df = df.copy()
    
    contract_df = contract_df[contract_df.clause.str.split().str.len() > 5]
    contract_df["clause"] = contract_df["clause"].str.replace('\n', ' ')
    contract_df = contract_df.replace(r'[^a-zA-Z ]', ' ', regex=True)
    contract_df = contract_df[contract_df['clause'] != '']
    contract_df = contract_df[~contract_df['clause'].str.isupper()]

    contract_df['tokens'] = contract_df['clause'].apply(tokenize)
    
    contract_df = contract_df[contract_df.tokens.str.split().str.len() > 5]
    
    contract_df.reset_index(drop=True, inplace=True)
    
    return contract_df


def remove_definitions(df):
    clean_df = df.copy()
    
    drop_index = []
    for index, row in df.iterrows():
        if len(list(get_definitions(row['clause']))) > 0:
            drop_index.append(index)
    
    if drop_index:
        clean_df = df.drop(drop_index)
        clean_df.reset_index(drop=True, inplace=True)
    
    return clean_df


def remove_signature(df):
    clean_df = df.copy()
    
    drop_index = -1
    for index, row in df.iterrows():
        if row['clause'].lower().startswith('in witness ') or 'WITNESS ' in row['clause'] or 'signature ' in row['clause'].lower() or 'signatures ' in row['clause'].lower():
            drop_index = index
            break
    
    if drop_index != -1:
        drop_indices = list(range(drop_index, df.index[-1]))
        clean_df = df.drop(drop_indices)
        clean_df.reset_index(drop=True, inplace=True)
    
    return clean_df


def read_contract(filename):
    contract_doc = Document(filename)

    record = []
    for para in contract_doc.paragraphs:
        if not para.text.lower().startswith("ex-"):
            record.append([para.text])

    contract_df = pd.concat([pd.DataFrame(columns=['clause']), 
                             pd.DataFrame(record, columns=['clause'])], ignore_index=True)
    
    contract_df = remove_definitions(contract_df)
    
    contract_df = remove_signature(contract_df)
    
    return preprocessing_contract(contract_df)
def print_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):
        display(df)

CUAD_to_kronicle = {'Governing Law': 'Governing Law', 
                    'Non Compete': 'Non Compete Clause', 
                    'No Solicit Of Customers': 'Non Solicitation',
                    'Termination For Convenience': 'Termination for Convenience',
                    'Post Termination Services': 'Effects of Termination',
                    'Anti Assignment': 'Anti Assignment',
                    'Audit Rights': 'Right to Audit',
                    'Cap On Liability': 'Limitation to Liability',
                   }

kronicle_to_CUAD = {'Termination for Convenience': 'Termination For Convenience',
                    'Effects of Termination': 'Post Termination Services',
                    'Limitation to Liability': 'Cap On Liability',
                    'Anti Assignment': 'Anti Assignment',
                    'Governing Law': 'Governing Law',
                    'Right to Audit': 'Audit Rights',
                    'Non Compete Clause': 'Non Compete',
                    'Non Solicitation': 'No Solicit Of Customers'
                   }
        

In [ ]:
file_path = "/content/drive/MyDrive/cuad-main/standard_clauses_updated.docx"
std_cls_doc = Document(file_path)

In [ ]:
std_cls_df = pd.DataFrame(columns=['category','sub_category','clause'])

doc_columns = ['sno', 'category','buy_cls','sell_cls']
doc_df = pd.DataFrame(columns=doc_columns)

for table in std_cls_doc.tables:
    for row in table.rows:
        record = [[value.text for value in row.cells]]
        doc_df = pd.concat([doc_df, pd.DataFrame(record, columns=doc_columns)], ignore_index=True)

doc_df = doc_df.drop([0, 1])
doc_df.reset_index(drop=True, inplace=True)

doc_df = doc_df.replace('“', '"', regex=True)
doc_df = doc_df.replace('”', '"', regex=True)

In [ ]:
std_cls_df = pd.DataFrame(columns=['category','sub_category','clause'])

for index, row in doc_df.iterrows():
    # processing BUY side clauses
    buy = row['buy_cls']
    buy_sub_category = re.findall('"([^"]*)"', buy)
    buy_record = []
    
    # handle empty rows
    if not buy_sub_category:
        buy_record.append([row['category'], row['category'], buy])
    else:
        if len(buy_sub_category) == 1:
            buy_record.append([row['category'], buy_sub_category[0], " ".join(buy.split('\n\n')[1:])])
        else:
            clauses = buy.split("\n\n")
            if len(clauses) == len(buy_sub_category):
                for i in range(len(clauses)):
                    buy_record.append([row['category'], buy_sub_category[i], clauses[i]])
            else:
                for i in range(0, len(clauses)):
                    text = clauses[i]
                    if text.endswith('"') and text.startswith('"'):
                        idx = i+1
                        full_cls = []
                        while idx < len(clauses) and not clauses[idx].endswith('"') and not clauses[idx].startswith('"'):
                            full_cls.append(clauses[idx])
                            idx+=1
                        
                        buy_record.append([row['category'], text.replace('"', ''), " ".join([val for val in full_cls])])
                    
                    elif text.startswith('"') and not text.endswith('"'):
                        sub_category = re.findall('"([^"]*)"', text)
                        buy_record.append([row['category'], sub_category[0], text])
                        
    std_cls_df = pd.concat([std_cls_df, pd.DataFrame(buy_record, columns=std_cls_df.columns)], ignore_index=True)
    
    
    # processing SELL side clauses
    sell = row['sell_cls']
    sell_sub_category = re.findall('"([^"]*)"', sell)
    sell_record = []
    
    # handle empty rows
    if not sell_sub_category:
        sell_record.append([row['category'], row['category'], sell])
    else:
        if len(sell_sub_category) == 1:
            sell_record.append([row['category'], sell_sub_category[0], " ".join(sell.split('\n\n')[1:])])
        else:
            clauses = sell.split("\n\n")
            if len(clauses) == len(sell_sub_category):
                for i in range(len(clauses)):
                    sell_record.append([row['category'], sell_sub_category[i], clauses[i]])
            else:
                for i in range(0, len(clauses)):
                    text = clauses[i]
                    if text.endswith('"') and text.startswith('"'):
                        idx = i+1
                        full_cls = []
                        while idx < len(clauses) and not clauses[idx].endswith('"') and not clauses[idx].startswith('"'):
                            full_cls.append(clauses[idx])
                            idx+=1
                        
                        sell_record.append([row['category'], text.replace('"', ''), " ".join([val for val in full_cls])])
                    
                    elif text.startswith('"') and not text.endswith('"'):
                        sub_category = re.findall('"([^"]*)"', text)
                        sell_record.append([row['category'], sub_category[0], text])
    
    std_cls_df = pd.concat([std_cls_df, pd.DataFrame(sell_record, columns=std_cls_df.columns)], ignore_index=True)


std_cls_df = preprocessing_master(std_cls_df)

In [ ]:
__author__ = "ContraxSuite, LLC; LexPredict, LLC"
__copyright__ = "Copyright 2015-2021, ContraxSuite, LLC"
__license__ = "https://github.com/LexPredict/lexpredict-lexnlp/blob/2.2.1.0/LICENSE"
__version__ = "2.2.1.0"
__maintainer__ = "LexPredict, LLC"
__email__ = "support@contraxsuite.com"


from typing import Generator, List, Tuple

from lexnlp.extract.common.annotation_locator_type import AnnotationLocatorType
from lexnlp.extract.common.annotations.definition_annotation import DefinitionAnnotation
from lexnlp.extract.en.definition_parsing_methods import DefinitionCaught, get_definition_list_in_sentence, \
    filter_definitions_for_self_repeating
from lexnlp.extract.ml.en.definitions.layered_definition_detector import LayeredDefinitionDetector
from lexnlp.nlp.en.segments.sentences import get_sentence_span


def get_definitions_in_sentence(sentence: str,
                                return_sources=False,
                                decode_unicode=True) -> Generator:
    definitions = get_definition_list_in_sentence((0, len(sentence), sentence), decode_unicode)
    for df in definitions:
        if return_sources:
            yield df.name, df.text
        else:
            yield df.name


def get_definition_objects_list(text, decode_unicode=True) -> List[DefinitionCaught]:
    """
    :param text: text to search for definitions
    :param decode_unicode:
    :return: a list of found definitions - objects of class DefinitionCaught
    """
    definitions = []
    for sentence in get_sentence_span(text):  # type: Tuple[int, int, str]
        definitions += get_definition_list_in_sentence(sentence, decode_unicode)
    definitions = filter_definitions_for_self_repeating(definitions)
    return definitions


parser_ml_classifier = LayeredDefinitionDetector()


def get_definition_annotations(text: str,
                               **kwargs) \
        -> Generator[DefinitionAnnotation, None, None]:
    decode_unicode = kwargs.get('decode_unicode', True)
    locator_type = kwargs.get('locator_type', AnnotationLocatorType.RegexpBased)

    if locator_type == AnnotationLocatorType.MlWordVectorBased:
        if not parser_ml_classifier.initialized:
            raise Exception('"parser_ml_classifier" object should be initialized (call load_compressed method)')
        ants = parser_ml_classifier.get_annotations(text)
        for ant in ants:
            yield ant
        return

    # use Regexp-based locator
    for d in get_definition_objects_list(
            text, decode_unicode=decode_unicode):
        ant = DefinitionAnnotation(
            coords=d.coords, text=d.text, name=d.name)
        yield ant


def get_definitions(text: str,
                    return_sources=False,
                    decode_unicode=True,
                    return_coords=False,
                    locator_type: AnnotationLocatorType = AnnotationLocatorType.RegexpBased) -> Generator:
    """
    Find possible definitions in natural language in text.
    The text will be split to sentences first.
    :param return_coords: returns a (x, y) tuple in each record. x - definition's text start, y - definition's text end
    :param decode_unicode:
    :param return_sources: returns a tuple with the extracted term and the source sentence
    :param text: the input text
    :param locator_type: use default (Regexp-based) or ML-based locator
    :return: Generator[name] or Generator[name, text] or Generator[name, text, coords]
    """
    if locator_type == AnnotationLocatorType.MlWordVectorBased:
        if not parser_ml_classifier.initialized:
            raise Exception('"parser_ml_classifier" object should be initialized (call load_compressed method)')
        definitions = parser_ml_classifier.get_annotations(text)
    else:
        definitions = get_definition_objects_list(text, decode_unicode)

    for df in definitions:
        if return_coords:
            yield df.name, df.text, (df.coords[0], df.coords[1])
        elif return_sources:
            yield df.name, df.text
        else:
            yield df.name


def get_definitions_explicit(text,
                             decode_unicode=True,
                             locator_type: AnnotationLocatorType = AnnotationLocatorType.RegexpBased) -> Generator:
    yield from get_definitions(text,
                               return_sources=True,
                               decode_unicode=decode_unicode,
                               return_coords=True,
                               locator_type=locator_type)

In [ ]:
import os
 
# Get the list of all files and directories
path = "/content/AI-Contract-Review/contracts/"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

Files and directories in ' /content/AI-Contract-Review/contracts/ ' :
['191120.SOG.HempExtracts.SupplyAgmt.docx', '191101.SOG.FoodServiceMarkets.SupplyAgmt.docx', '000000.SOG.Goods.MasterSupplyAgmt.docx', '191127.SOG.HempBiomass.SupplyAgmt.docx', '200817.SOG.COV-19Vaccines.SupplyAgmt.docx', '021024.SOG.Tiles.SupplyAgreement.docx', '201500.SOG.SoftwareProducts.SupplyAgmt.docx', '980810.SOG.FlashMemoryDevice.MasterPurchaseAgreement.docx', '190207.SOG.Products.PurchaseAgmt.docx', '000323.SOG.LowAlkaliProduct.SupplyAgreement.docx', 'SOG.Equipment.AgreementforSaleofEquipment.docx', '010400.SOG.CoffeeProducts.SupplyAgmt.docx', '200203.SOG.PharmaceuticalProducts.SupplyAgmt.docx', '181213.SOG.SaleOfRehabilitationRobots.SaleOfGoodsAgmt.docx', '000000.SOG.WaterTreatmentEquipment.SupplyAgreement.docx']


In [ ]:
contract_df1 = read_contract(path+"000000.SOG.WaterTreatmentEquipment.SupplyAgreement.docx")

In [ ]:
contract_df = [read_contract(path+contract_name) for contract_name in dir_list]

In [ ]:
contract_df1

,clause,tokens
0,A Culligan is engaged in the business of...,culligan engage business sell type water treat...
1,B Culligan desires to provide certain pr...,b culligan desire provide certain product buye...
2,NOW THEREFORE for good and valuable cons...,good valuable consideration receipt sufficienc...
3,ADDITIONAL TERMS CONDITIONS Any add...,additional terms condition additional term con...
4,LIMITATION ON LIABILITY Notwithstandi...,limitation liability notwithstanding contrary ...
5,a If either party fails to perform...,party fail perform material respect non moneta...
6,b If Buyer fails to pay any amount...,b buyer fail pay amount agreement receive writ...
7,c If a either party defaults under...,c party default subsection b twice month period
8,e If either party becomes insolven...,e party insolvent unable pay debt ordinary cou...
9,f If a either party ceases to func...,f party cease function go concern conduct oper...


In [ ]:
std_cls_df

,category,sub_category,clause,tokens
0,Operative Clauses,Delivery Date and Delivery Location Buy side,Seller shall deliver the Goods on the Delivery...,seller shall deliver good delivery date specif...
1,Operative Clauses,Delivery Date and Delivery Location Sell side,Seller shall deliver the Goods within a reason...,seller shall deliver good reasonable time date...
2,Operative Clauses,Quantity Buy side,Seller shall deliver the quantities of the Goo...,seller shall deliver quantity goods specify ag...
3,Operative Clauses,Quantity Sell side,If Seller delivers more than or less than...,seller deliver quantity goods specify annexure...
4,Operative Clauses,Shipping Terms Buy side,Delivery of the Goods shall be made in accorda...,delivery good shall accordance term set forth ...
...,...,...,...,...
81,Term,Term,This Agreement shall be effective from Effecti...,agreement shall effective effective date shall...
82,Third Party Rights,Third Party Rights,Unless explicitly stated elsewhere in this Agr...,explicitly state agreement agreement shall con...
83,Compliance with Law,Compliance with Law,Each Party will comply with all Applicable Law...,party comply applicable laws performance oblig...
84,Cost Fee,Cost Fee,Subject to any express provisions to the contr...,subject express provision contrary agreement p...


In [ ]:
std_cls_df.describe()

,category,sub_category,clause,tokens
count,86,86,86,86
unique,38,85,86,85
top,Operative Clauses,Entire Agreement,Seller shall deliver the Goods on the Delivery...,buyer shall purchase good seller price set for...
freq,21,2,1,2


In [ ]:
#contract_df[1]

In [ ]:
std_cls_df["tokens"][1]

'seller shall deliver good reasonable time date agreement subject availability finished good time quote seller delivery estimate good shall deliver address specify agreement set forth annexure purchase order'

In [ ]:
std_cls_df['category'][75]

'Further Assurance '

In [ ]:
std_cls_df['sub_category'][51]

'Limitation to Liability  Buy side'

In [ ]:
text_data = [para.split() for para in std_cls_df["tokens"]]

Doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

In [ ]:
def tagged_document(list_of_list_of_words):
  for i, list_of_words in enumerate(list_of_list_of_words):
    yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

data_for_training = list(tagged_document(text_data))

In [ ]:
len(data_for_training)

86

In [ ]:
print(data_for_training[:1])

[TaggedDocument(words=['seller', 'shall', 'deliver', 'good', 'delivery', 'date', 'specify', 'agreement', 'set', 'forth', 'annexure', 'purchase', 'order', 'good', 'shall', 'deliver', 'address', 'specify', 'agreement', 'set', 'forth', 'annexure', 'purchase', 'order', 'buyer', 's', 'normal', 'business', 'hour', 'instruct', 'buyer'], tags=[0])]


In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=50)
model.build_vocab(data_for_training)

In [ ]:
model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
data_for_training[1]

TaggedDocument(words=['seller', 'shall', 'deliver', 'good', 'reasonable', 'time', 'date', 'agreement', 'subject', 'availability', 'finished', 'good', 'time', 'quote', 'seller', 'delivery', 'estimate', 'good', 'shall', 'deliver', 'address', 'specify', 'agreement', 'set', 'forth', 'annexure', 'purchase', 'order'], tags=[1])

In [ ]:
tc = '              Notices  All approvals and notices under this Agreement must be in writing and shall be deemed to have been duly given or made  i  the third business day after the date of mailing  if delivered by registered or certified mail  postage prepaid   ii  upon delivery  if sent by hand delivery   iii  upon delivery  if sent by prepaid courier  with a record of receipt  or  iv  the next day after the date of'.split()

In [ ]:
tc1 = contract_df1["tokens"][3]
tc1

'additional terms condition additional term condition conflict term condition offer buyer purchase order acknowledgement thereto manner shall void term agree writing party case conflict term condition attachment provision agreement provision agreement shall control order accept culligan constitute binding obligation agreement culligan sell buyer buy pay products order'

In [ ]:
inferred_vector = model.infer_vector(tc1.split())
rankings = model.dv.most_similar([inferred_vector])
rankings

[(36, 0.8549333810806274),
 (35, 0.8464059829711914),
 (51, 0.8327101469039917),
 (14, 0.809186577796936),
 (52, 0.8079453706741333),
 (5, 0.8004401922225952),
 (60, 0.7971333861351013),
 (43, 0.7899860739707947),
 (34, 0.7754584550857544),
 (26, 0.7718598246574402)]

In [ ]:
ranks = [(std_cls_df['sub_category'][rank[0]],rank[1]) for rank in rankings]

In [ ]:
ranks

[('Termination for Material Breach   Sell side', 0.8549333810806274),
 ('Termination for Material Breach  Buy side', 0.8464059829711914),
 ('Limitation to Liability  Buy side', 0.8327101469039917),
 ('Performance Guarantee  Buy side', 0.809186577796936),
 ('Limitation to Liability   Sell side', 0.8079453706741333),
 ('Shipping Terms   Sell side', 0.8004401922225952),
 ('Severability', 0.7971333861351013),
 ('DISCLAIMER  Buy side', 0.7899860739707947),
 ('Termination for Convenience   Sell side', 0.7754584550857544),
 ('No Set Off   Sell side', 0.7718598246574402)]

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

def tagged_document(list_of_list_of_words):
  for i, list_of_words in enumerate(list_of_list_of_words):
    yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

def DOC2VEC(std_cls_df,contract_df):
  text_data = [para.split() for para in std_cls_df["tokens"]]
  data_for_training = list(tagged_document(text_data))
  model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=50)
  model.build_vocab(data_for_training)
  model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)

  test_clauses = [para for para in contract_df1["tokens"]]
  inferred_vector = [model.infer_vector(clause.split()) for clause in test_clauses]
  rankings = [model.dv.most_similar([vectors],topn=5) for vectors in inferred_vector]
  scores = []
  for ranks in rankings:
    rank1 = [(std_cls_df['sub_category'][rank[0]],rank[1]) for rank in ranks]
    scores.append(rank1)

  return scores

In [ ]:
res = DOC2VEC(std_cls_df,contract_df1)

In [ ]:
cuad_std_cls_df = pd.read_excel('/content/AI-Contract-Review/CUAD_clauses.xlsx', index_col=None)
cuad_std_cls_df['category'] = cuad_std_cls_df['category'].replace(r'[^a-zA-Z ]', ' ', regex=True)
cuad_std_cls_df = cuad_std_cls_df[cuad_std_cls_df['category'].isin(list(CUAD_to_kronicle.keys()))]
cuad_std_cls_df.reset_index(drop=True, inplace=True)

In [ ]:
cuad_std_cls_df

,category,question,kronicle
0,Governing Law,Highlight the parts (if any) of this contract ...,Governing Law
1,Non Compete,Highlight the parts (if any) of this contract ...,Non Compete Clause
2,No Solicit Of Customers,Highlight the parts (if any) of this contract ...,Non Solicitation
3,Termination For Convenience,Highlight the parts (if any) of this contract ...,Termination for Convenience
4,Anti Assignment,Highlight the parts (if any) of this contract ...,Anti Assignment
5,Post Termination Services,Highlight the parts (if any) of this contract ...,Effects of Termination
6,Audit Rights,Highlight the parts (if any) of this contract ...,Right to Audit
7,Cap On Liability,Highlight the parts (if any) of this contract ...,Limitation to Liability


## PyTerrier

In [ ]:
if not pt.started():
    pt.init()

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [ ]:
assert pt.__version__ > '0.8', "PyTerrier 0.8.0 or newer is required"

In [ ]:
top_categories = {'Indemnification': ['Indemnity', 'Intellectual Property Rights Indemnity'], 
                  'Termination': ['Termination for Cause', 'Termination for Convenience', 'Termination for Material Breach'], 
                  'Effects of Termination': ['Effects of Termination'], 
                  'Limitation of Liability': ['Limitation on Amount', 'Limitation to Liability'], 
                  'Disclaimer': ['DISCLAIMER'], 
                  'Confidential Information': ['Confidential Information'], 
                  'Intellectual Property Rights': ['Intellectual Property Rights'], 
                  'Operative Clauses': ['Late Delivery and Penalty', 'Performance Guarantee', 'Insurance']
                 }

top_sub_categories = ['Indemnity', 'Intellectual Property Rights Indemnity', 
                     'Termination for Cause', 'Termination for Convenience', 'Termination for Material Breach', 
                     'Effects of Termination', 
                     'Limitation on Amount', 'Limitation to Liability', 
                     'DISCLAIMER', 
                     'Confidential Information', 
                     'Intellectual Property Rights', 
                     'Late Delivery and Penalty', 'Performance Guarantee', 'Insurance']


def get_missing_categories(df, groups):
    return set(df.docno).difference(set(groups.groups.keys()))


def get_top_missing_categories(df, groups):
    all_categories = get_missing_categories(df, groups)
    
    missing_df = pd.DataFrame(columns=['category', 'sub_category'])
    record = []
    
    for category in all_categories:
        for key, value in top_categories.items():
            if category in value:
                record.append([key, category])
    
    missing_df = pd.concat([missing_df, pd.DataFrame(record, columns=missing_df.columns)], ignore_index=True)
    return missing_df


def missing_category_results(master_df, index, queries):
    missing = {}
    for key, value in similarity_models(index, queries).items():
        df = value.copy()
        df = df[df['rank'] < 5]
        missing[key] = get_top_missing_categories(master_df, df.groupby(by='docno'))
        missing[key].columns = pd.MultiIndex.from_product([[key], missing[key].columns])
    
    return pd.concat(missing.values(), axis = 1)



def create_queries(contract_df):
    queries_df = contract_df.copy()
    queries_df['qid'] = ['q' + str(i+1) for i in queries_df.index.values]
    queries_df = queries_df.rename(columns={"tokens": "query"})
    return queries_df


def convert(number):  
    return str(round(number, 2))


def join(value):
    df = value.copy()
    df['score'] = df['score'].apply(convert)
    df = df.apply(' : '.join, axis=1).reset_index()
    df.rename({0: 'docno'}, inplace=True, axis='columns')
    return ' \n '.join(df['docno'])

def categories_to_excel(filename, sheetname, results): 
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    results.to_excel(writer, sheet_name=sheetname, startrow=1 , startcol=0, header=False, index=False, encoding='utf8')  
    workbook  = writer.book
    worksheet = writer.sheets[sheetname]

    format_header = workbook.add_format()
    format_header.set_align('center')
    format_header.set_bold()
    format_header.set_text_wrap()
    format_header.set_border()

    format_data = workbook.add_format()
    format_data.set_text_wrap()

    worksheet.set_column('A:A', 10, format_data)
    worksheet.set_column('B:F', 40, format_data)
    worksheet.set_row(1, 150, format_data)
    worksheet.set_row(0, 20, format_header)

    # Write the header manually
    for colx, value in enumerate(results.columns.values):
        worksheet.write(0, colx, value)

    writer.save()

In [ ]:
def run_queries(index, queries):
    results = pd.DataFrame(columns=['clauseid', 'clause'])
    for key, value in similarity_models(index, queries).items():
        results = model_results(value.groupby(by='qid'), results, key)
    #print(results['TF_IDF'][3])
    return results

def similarity_models(index, queries, measures=['TF_IDF', 'BM25', 'PL2', 'Hiemstra_LM', 'DirichletLM']):
    result_measures = {}
    for measure in measures:
        result_measures[measure] = similarity_model(index, measure, queries)
    #print(result_measures)
    return result_measures
    


def similarity_model(index, weighting_model, queries):
    batch_ret = pt.BatchRetrieve(index, wmodel=weighting_model)
    results = batch_ret.transform(queries)
    #print(results.head())
    #categories_to_excel('Results/Contract 1 Categories_old.xlsx', 'Contract 1', results)
    return results


def model_results(groups, result, model):
    record = []
    for group in groups.groups:
        df = groups.get_group(group)
        df = df.drop_duplicates(subset=['docno'])
        #df = df.head()
        df = df.drop(columns=['docid', 'rank'])
        df = df.groupby(by=['qid', 'clause', 'query'])[['docno', 'score']].apply(join).reset_index()
        df.rename({0: 'docno'}, inplace=True, axis='columns')

        for index, row in df.iterrows():
            record.append([row['qid'], row['clause'], row['docno']])

    measure = pd.DataFrame(record, columns=['clauseid', 'clause', model])
    results = result.merge(measure, how='outer', on=["clauseid", 'clause'])
    return results


In [ ]:
queries1 = create_queries(contract_df1)
#print_df(std_cls_df)
#queries1

In [ ]:
master_df = std_cls_df.copy()
master_df = std_cls_df.rename(columns={"sub_category": "docno"})

indexer = pt.DFIndexer("./std_cls_idx", overwrite=True)
indexref = indexer.index(master_df["tokens"], master_df['docno'])

%mkdir Results
%mkdir Eval
%ls

AI-Contract-Review/  drive/  Eval/  Results/  sample_data/  std_cls_idx/


Running models against queries
Test doc 1

In [ ]:
results1 = run_queries(indexref, queries1)
#categories_to_excel('Results/Contract 1 Categories_old.xlsx', 'Contract 1', results1)

In [ ]:
k =[int(a[1:]) for a in results1['clauseid']]

In [ ]:
results1.insert(0,"S no.",k) 

In [ ]:
results1

,S no.,clauseid,clause,TF_IDF,BM25,PL2,Hiemstra_LM,DirichletLM
0,1,q1,A Culligan is engaged in the business of...,Non Compete Clause : 7.04 \n Force Majeure Eve...,Non Compete Clause : 11.39 \n Force Majeure Ev...,Non Compete Clause : 6.96 \n Force Majeure Eve...,Non Compete Clause : 7.25 \n Force Majeure Eve...,Force Majeure Event : 1.93 \n Non Compete Clau...
1,10,q10,f If a either party ceases to func...,Dispute Resolution : 6.13 \n Intellectual Prop...,Delivery Date and Delivery Location Buy side...,Delivery Date and Delivery Location Buy side...,Dispute Resolution : 4.92 \n Delivery Date and...,Dispute Resolution : 1.9 \n DISCLAIMER Sell ...
2,11,q11,g If Buyer assigns this Agreement ...,Anti Assignment : 11.75 \n Intellectual Proper...,Anti Assignment : 10.76 \n Intellectual Proper...,Anti Assignment : 10.38 \n Intellectual Proper...,Anti Assignment : 8.4 \n Intellectual Property...,Payment Terms Buy side : 1.57 \n Intellectual...
3,12,q12,Remedies Upon the occurrence of...,Inspection Buy side : 7.18 \n Default : 6.81 ...,Default : 8.01 \n Inspection Buy side : 7.44 ...,Inspection Buy side : 5.6 \n Default : 5.41 \...,Remedies Buy side : 4.78 \n Remedies Sell s...,Inspection Buy side : 3.8 \n Inspection Sel...
4,13,q13,Effect of Termination Terminati...,Survival : 5.23 \n Default : 5.03 \n Effects o...,Default : 3.06 \n Term : 2.19 \n Notice of For...,Effects of Termination Buy side : 4.11 \n Sur...,No Set Off Sell side : 3.78 \n Term : 3.48 \...,Force Majeure Event : 2.2 \n Default : 1.4 \n ...
5,14,q14,Assignment Except as provided i...,Anti Assignment : 13.93 \n Intellectual Proper...,Anti Assignment : 13.97 \n Intellectual Proper...,Anti Assignment : 12.34 \n Intellectual Proper...,Anti Assignment : 9.99 \n Intellectual Propert...,Payment Terms Buy side : 2.04 \n Intellectual...
6,15,q15,Relationship between the Parties...,Relationship : 12.67 \n Pre Inspection Sell ...,Relationship : 17.57 \n Pre Inspection Sell ...,Relationship : 11.25 \n Pre Inspection Sell ...,Relationship : 11.63 \n Pre Inspection Sell ...,Relationship : 6.48 \n Right to Audit : 1.88 \...
7,16,q16,Notices All approvals and notic...,Notice : 13.26 \n Right to make changes Buy s...,Notice : 20.19 \n Right to make changes Buy s...,Notice : 10.53 \n Right to make changes Buy s...,Notice : 8.75 \n Right to make changes Buy si...,Notice : 6.61 \n Right to make changes Buy si...
8,17,q17,dispatch if sent by cable telegram facsimil...,Notice : 18.84 \n Effects of Termination Sel...,Notice : 30.38 \n Effects of Termination Sel...,Notice : 14.69 \n Further Assurance : 5.87 \n ...,Notice : 13.82 \n Effects of Termination Sel...,Notice : 5.03 \n Effects of Termination Sell...
9,18,q18,Severability If any term or pro...,Severability : 11.91 \n Dispute Resolution : 4...,Severability : 18.12 \n Dispute Resolution : 6...,Severability : 11.4 \n Dispute Resolution : 3....,Severability : 10.72 \n Dispute Resolution : 3...,Severability : 6.58 \n Dispute Resolution : 3....


In [ ]:
results1.sort_values(by = 'S no.')

,S no.,clauseid,clause,TF_IDF,BM25,PL2,Hiemstra_LM,DirichletLM
0,1,q1,A Culligan is engaged in the business of...,Non Compete Clause : 7.04 \n Force Majeure Eve...,Non Compete Clause : 11.39 \n Force Majeure Ev...,Non Compete Clause : 6.96 \n Force Majeure Eve...,Non Compete Clause : 7.25 \n Force Majeure Eve...,Force Majeure Event : 1.93 \n Non Compete Clau...
11,2,q2,B Culligan desires to provide certain pr...,Further Assurance : 5.7 \n Effects of Terminat...,Further Assurance : 6.93 \n Packaging Sell s...,Further Assurance : 5.08 \n Effects of Termina...,Further Assurance : 5.69 \n Effects of Termina...,Further Assurance : 1.52 \n Effects of Termina...
19,3,q3,NOW THEREFORE for good and valuable cons...,Notice : 5.59 \n Confidential Information Buy...,Notice : 8.36 \n Confidential Information Buy...,Notice : 4.79 \n Equitable Remedies : 4.1 \n F...,Notice : 3.79 \n Further Assurance : 3.63 \n C...,Confidential Information Buy side : 1.4 \n Co...
20,4,q4,ADDITIONAL TERMS CONDITIONS Any add...,Severability : 7.37 \n Termination for Materia...,Severability : 6.77 \n Cancellation Fee Sell...,Severability : 6.93 \n Termination for Materia...,Severability : 5.37 \n Termination for Materia...,Force Majeure Event : 3.63 \n Severability : 2...
21,5,q5,LIMITATION ON LIABILITY Notwithstandi...,Limitation to Liability Buy side : 11.35 \n L...,Limitation to Liability Buy side : 17.13 \n L...,Limitation to Liability Buy side : 9.21 \n Li...,Limitation to Liability Buy side : 7.73 \n Li...,Limitation to Liability Buy side : 6.91 \n Li...
22,6,q6,a If either party fails to perform...,Default : 10.13 \n Opportunity to Cure Buy si...,Default : 14.36 \n Opportunity to Cure Buy si...,Default : 7.84 \n Equitable Remedies : 6.98 \n...,Default : 6.28 \n Equitable Remedies : 5.6 \n ...,Default : 2.73 \n Equitable Remedies : 2.5 \n ...
23,7,q7,b If Buyer fails to pay any amount...,Opportunity to Cure Buy side : 5.94 \n Notice...,Opportunity to Cure Buy side : 7.02 \n Defaul...,Notice : 5.27 \n Opportunity to Cure Buy side...,Opportunity to Cure Buy side : 3.6 \n Default...,Default : 1.81 \n Notice : 1.07 \n Payment Ter...
24,8,q8,c If a either party defaults under...,Default : 10.24 \n Consequences of a Force Maj...,Default : 15.07 \n Penalty for Late Delivery a...,Default : 8.21 \n Penalty for Late Delivery an...,Default : 6.48 \n Penalty for Late Delivery an...,Default : 1.84 \n Penalty for Late Delivery an...
25,9,q9,e If either party becomes insolven...,Quantity Sell side : 8.53 \n DISCLAIMER Se...,Quantity Sell side : 14.07 \n DISCLAIMER S...,Quantity Sell side : 7.01 \n DISCLAIMER Se...,Quantity Sell side : 7.34 \n DISCLAIMER Se...,Quantity Sell side : 2.13 \n Force Majeure E...
1,10,q10,f If a either party ceases to func...,Dispute Resolution : 6.13 \n Intellectual Prop...,Delivery Date and Delivery Location Buy side...,Delivery Date and Delivery Location Buy side...,Dispute Resolution : 4.92 \n Delivery Date and...,Dispute Resolution : 1.9 \n DISCLAIMER Sell ...


In [ ]:
results1.insert(3,"Doc2Vec",res) 

In [ ]:
results1["Doc2Vec"][2]

[('Anti Bribery', 0.9586225152015686),
 ('Packaging  Buy side', 0.9554973244667053),
 ('Waiver', 0.9537368416786194),
 ('Further Assurance ', 0.9513251185417175),
 ('Relationship ', 0.9482403993606567)]

In [ ]:

for j in range(len(results1["Doc2Vec"])):
  cd=''
  for i in results1["Doc2Vec"][j]:
    cd += str(i[0])+' : '+str(i[1])+' \n '
  results1["Doc2Vec"][j]=cd


<ipython-input-50-4a6e23fae08b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results1["Doc2Vec"][j]=cd


below code for ranking [borda count] (works good)

In [ ]:
import pandas as pd
#df = pd.read_excel('Results/Contract 1 Categories_old.xlsx')
df=results1
modl=['TF_IDF', 'BM25', 'PL2', 'Hiemstra_LM','DirichletLM']
y=[]
k=[]
t=[]
v=[]
#len(df['clause'])
for j in range(len(df['clause'])):
	p=[]
	for i in modl:
		p.append(df[i][j])
	y.append(p)

for j in y:
	q=[]
	for i in j:
		a=i.split(' \n ')
		q.append(list(a))
    #for l in a:
		#	q.append(l)
	k.append(q)
 
for j in k:
  s=[]
  for l in j:
    r=[]
    for i in l:
      r+=(i.split(" : "))
    s.append(r)
  t.append(s) 

for j in t:
  for k in j:
    num=1
    for i in range(len(k)):
      if i%2!=0:
        k[i]=int(num)
        num+=1


for k in range(len(t)):
  tst=t[k]
  ts=[]
  for i in tst:
    nu=0
    for j in range(int(len(i)/2)):
      lst=[i[nu], i[nu+1]]
      ts.append(lst)
      nu+=2
  t[k]=ts


for j in range(len(df['clause'])):
	df2=pd.DataFrame(t[j], columns =['clause', 'rank'])
	#df2.sort_values(by=['clause'])
	#df2.groupby(['clause'])['score'].apply(lambda x: ','.join(x.astype(str))).reset_index()
	df2=df2.groupby(['clause'])['rank'].apply(lambda x: sum(x)).reset_index()
	df2.sort_values(by=['rank'], inplace=True, ascending=[True])
	df2=df2.reset_index(drop=True)
	e=[]
	#for i in range(len(df2['score'])):
	for i in range(5):
		e.append(df2['clause'][i]+' : '+str(df2['rank'][i])+'\n')
  #f=[]
	v.append(''.join(e))	
	#v.append(f)

  
df['sorted by rank']=v

Below code is to sum the percent scores of all the models (works well)

In [ ]:
import pandas as pd
#df = pd.read_excel('Results/Contract 1 Categories_old.xlsx')
df=results1
#modl=['TF_IDF', 'BM25', 'PL2', 'Hiemstra_LM','DirichletLM']
y=[]
k=[]
t=[]
v=[]
#len(df['clause'])
for j in range(len(df['clause'])):
	p=[]
	for i in modl:
		p.append(df[i][j])
	y.append(p)

for j in y:
	q=[]
	for i in j:
		a=i.split(' \n ')
		#for l in a:
		#	q.append(l)
		q.append(list(a))
	k.append(q)

for j in k:
  s=[]
  for l in j:
    r=[]
    for i in l:
      r+=(i.split(" : "))
    s.append(r)
  t.append(s) 

for j in t:
  for k in j:
    #num=1
    su=[]
    for i in range(len(k)):
      if i%2!=0:
        k[i]=float(k[i])
        su.append(k[i])
        #num+=1

    num=1
    for i in range(int(len(k)/2)):
      k[num]=k[num]*100/sum(su)
      k[num]=round(k[num],2)
      num+=2

for k in range(len(t)):
  tst=t[k]
  ts=[]
  for i in tst:
    nu=0
    for j in range(int(len(i)/2)):
      lst=[i[nu], i[nu+1]]
      ts.append(lst)
      nu+=2
  t[k]=ts

for j in range(len(df['clause'])):
	df2=pd.DataFrame(t[j], columns =['clause', 'score'])
	#df2.sort_values(by=['clause'])
	#df2.groupby(['clause'])['score'].apply(lambda x: ','.join(x.astype(str))).reset_index()
	df2=df2.groupby(['clause'])['score'].apply(lambda x: sum(x)).reset_index()
	df2.sort_values(by=['score'], inplace=True, ascending=[False])
	df2=df2.reset_index(drop=True)
	e=[]
	#for i in range(len(df2['score'])):
	for i in range(5):
		e.append(df2['clause'][i]+' : '+str(df2['score'][i])+'\n')
	#f=[]
	v.append(''.join(e))	  
	#v.append(f)
 
df['sorted by score']=v

Below code is to sum the scores of all the models (works well)

In [ ]:
import pandas as pd
#df = pd.read_excel('Results/Contract 1 Categories_old.xlsx')
df=results1
modl=['TF_IDF', 'BM25', 'PL2', 'Hiemstra_LM','DirichletLM']
y=[]
k=[]
t=[]
v=[]
#len(df['clause'])
for j in range(len(df['clause'])):
	p=[]
	for i in modl:
		p.append(df[i][j])
	y.append(p)

for j in y:
	q=[]
	for i in j:
		a=i.split(' \n ')
		#for l in a:
		#	q.append(l)
		q.append(list(a))
	k.append(q)

for j in k:
  s=[]
  for l in j:
    r=[]
    for i in l:
      r+=(i.split(" : "))
    s.append(r)
  t.append(s) 

for j in t:
  for k in j:
    #num=1
    #su=[]
    for i in range(len(k)):
      if i%2!=0:
        k[i]=float(k[i])
        #su.append(k[i])
        #num+=1

#    num=1
#    for i in range(int(len(k)/2)):
#      k[num]=k[num]*100/sum(su)
#      k[num]=round(k[num],2)
#      num+=2

for k in range(len(t)):
  tst=t[k]
  ts=[]
  for i in tst:
    nu=0
    for j in range(int(len(i)/2)):
      lst=[i[nu], i[nu+1]]
      ts.append(lst)
      nu+=2
  t[k]=ts

buyorsell = []

for j in range(len(df['clause'])):
	df2=pd.DataFrame(t[j], columns =['clause', 'score_sum'])
	#df2.sort_values(by=['clause'])
	#df2.groupby(['clause'])['score'].apply(lambda x: ','.join(x.astype(str))).reset_index()
	df2=df2.groupby(['clause'])['score_sum'].apply(lambda x: sum(x)).reset_index()
	df2.sort_values(by=['score_sum'], inplace=True, ascending=[False])
	df2=df2.reset_index(drop=True)
	e=[]
	#for i in range(len(df2['score'])):
	for i in range(5):
		e.append(df2['clause'][i]+' : '+str(df2['score_sum'][i])+'\n')
		if i == 0:
			buyorsell.append(df2['clause'][i])
	#f=[]
	v.append(''.join(e))	  
	#v.append(f)
 
df['sorted by score_sum']=v

In [ ]:
categories_to_excel('Results/Contract 1 Scores_sorted.xlsx', 'Contract 1', df)

In [ ]:
#df.to_csv('file1.csv')

In [ ]:
counter = 0
s = "sell"
for i in range(len(buyorsell)):
  if s in buyorsell[i].lower():
    counter+=1
if counter > len(buyorsell)/2:
  df = df.style.set_caption("Sell Side Contract")
else:
  df = df.style.set_caption("Buy Side Contract")

In [ ]:
df

,S no.,clauseid,clause,Doc2Vec,TF_IDF,BM25,PL2,Hiemstra_LM,DirichletLM,sorted by rank,sorted by score,sorted by score_sum
0,1,q1,A Culligan is engaged in the business of selling various types of water treatment products and,Non Solicitation : 0.9725136756896973 Relationship : 0.9661207795143127 Anti Bribery : 0.9623964428901672 Non Disparagement : 0.9619945883750916 Non Compete Clause : 0.9577245116233826,Non Compete Clause : 7.04 Force Majeure Event : 3.58 Pre Inspection Sell side : 2.15 Non Disparagement : 2.1 Delivery Date and Delivery Location Buy side : 2.05 Cancellation Fee Sell side : 1.94 Opportunity to Cure Buy side : 1.9 Notice : 1.89 Effects of Termination Buy side : 1.73 Quantity Sell side : 1.47 Shipping Terms Buy side : 1.41 Inspection Sell side : 1.34 Confidential Information Buy side : 1.27 Confidential Information Sell side : 1.27 Payment Terms Sell side : 1.13 Intellectual Property Rights Buy side : 0.97 Intellectual Property Rights Sell side : 0.97,Non Compete Clause : 11.39 Force Majeure Event : 6.02 Pre Inspection Sell side : 3.34 Cancellation Fee Sell side : 3.33 Non Disparagement : 3.26 Delivery Date and Delivery Location Buy side : 3.2 Effects of Termination Buy side : 2.97 Opportunity to Cure Buy side : 2.96 Notice : 2.95 Quantity Sell side : 2.29 Shipping Terms Buy side : 2.2 Confidential Information Buy side : 2.17 Confidential Information Sell side : 2.17 Inspection Sell side : 2.09 Payment Terms Sell side : 1.77 Intellectual Property Rights Buy side : 1.52 Intellectual Property Rights Sell side : 1.5,Non Compete Clause : 6.96 Force Majeure Event : 2.92 Pre Inspection Sell side : 1.81 Non Disparagement : 1.73 Delivery Date and Delivery Location Buy side : 1.68 Cancellation Fee Sell side : 1.64 Notice : 1.52 Opportunity to Cure Buy side : 1.5 Effects of Termination Buy side : 1.43 Quantity Sell side : 1.08 Shipping Terms Buy side : 1.03 Inspection Sell side : 0.96 Confidential Information Buy side : 0.91 Confidential Information Sell side : 0.91 Payment Terms Sell side : 0.78 Intellectual Property Rights Buy side : 0.64 Intellectual Property Rights Sell side : 0.63,Non Compete Clause : 7.25 Force Majeure Event : 4.19 Cancellation Fee Sell side : 1.86 Effects of Termination Buy side : 1.67 Non Disparagement : 1.51 Confidential Information Buy side : 1.47 Confidential Information Sell side : 1.47 Delivery Date and Delivery Location Buy side : 1.43 Pre Inspection Sell side : 1.31 Opportunity to Cure Buy side : 1.2 Notice : 1.03 Quantity Sell side : 0.75 Shipping Terms Buy side : 0.71 Inspection Sell side : 0.65 Payment Terms Sell side : 0.51 Intellectual Property Rights Buy side : 0.41 Intellectual Property Rights Sell side : 0.41,Force Majeure Event : 1.93 Non Compete Clause : 1.89 Confidential Information Buy side : 0.77 Confidential Information Sell side : 0.77 Cancellation Fee Sell side : 0.6 Effects of Termination Buy side : 0.58 Pre Inspection Sell side : 0.23 Notice : 0.22 Non Disparagement : 0.12 Delivery Date and Delivery Location Buy side : 0.12 Opportunity to Cure Buy side : 0.12 Quantity Sell side : 0.1 Shipping Terms Buy side : 0.1 Inspection Sell side : 0.09 Payment Terms Sell side : 0.08 Intellectual Property Rights Buy side : 0.06 Intellectual Property Rights Sell side : 0.06,Non Compete Clause : 6 Force Majeure Event : 9 Cancellation Fee Sell side : 24 Pre Inspection Sell side : 25 Non Disparagement : 27,Non Compete Clause : 116.13999999999999 Force Majeure Event : 71.44000000000001 Cancellation Fee Sell side : 31.869999999999997 Effects of Termination Buy side : 28.93 Pre Inspection Sell side : 26.41,Non Compete Clause : 34.53 Force Majeure Event : 18.64 Cancellation Fee Sell side : 9.37 Pre Inspection Sell side : 8.840000000000002 Non Disparagement : 8.719999999999999
1,10,q10,f If a either party ceases to function as a going concern or to conduct its operations in the normal course of business or,Non Solicitation : 0.9601502418518066 Survival : 0.9499334096908569 Relationship : 0.947